In [1]:
!apt update && apt install -y curl git

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,315 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,655 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [31]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
#######                                                                                         8.4%^C


In [42]:
!nohup ollama serve > /dev/null 2>&1 &


In [34]:
!ollama pull llama2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 8934d96d3f08... 100% ▕▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕▏ 4.8 KB                         
pulling 2e0493f67d0c... 100% ▕▏   59 B                         
pulling fa304d675061... 100% ▕▏   91 B                         
pulling 42ba7f8a01dd... 100% ▕▏  557 B                         
verifying sha256 digest 
writing manifest 
success 


In [43]:
OLLAMA_API_URL = "http://127.0.0.1:11434"

In [44]:
!pkill -f ngrok  # Kills all ngrok processes
!fuser -k 5000/tcp  # Kills any process using port 6000 (or use 5000 if needed)
# Install required libraries

In [45]:
!pip install requests PyPDF2


In [ ]:
import requests
import PyPDF2
import io
from google.colab import files

# Ollama API URL (Replace with your local Ollama ngrok URL if required)
OLLAMA_API_URL = "http://localhost:11434/api/generate"

# Function to upload a PDF resume
def upload_pdf():
    uploaded = files.upload()
    pdf_filename = list(uploaded.keys())[0]
    return pdf_filename

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
            return text.strip()
    except Exception as e:
        return f"Error extracting text: {str(e)}"

# Step 1: Upload Resume
print("📂 Please upload your resume (PDF format)...")
pdf_path = upload_pdf()

# Step 2: Extract Resume Text
resume_text = extract_text_from_pdf(pdf_path)

if not resume_text:
    print("❌ Error: Could not extract text from the uploaded PDF.")
else:
    print("\n✅ Resume Text Extracted Successfully!")

# Step 3: Get Job Description Input
job_description = input("\n📌 Enter the Job Description: ").strip()

# Step 4: Prepare Ollama Prompt
ollama_prompt = (
    f"Compare the following resume with the given job description:\n\n"
    f"Resume:\n{resume_text}\n\n"
    f"Job Description:\n{job_description}\n\n"
    f"Perform the following analysis:\n"
    f"1. Extract the programming languages mentioned in both the resume and job description.\n"
    f"2. Extract the soft skills (e.g., communication, teamwork, problem-solving, leadership) mentioned in both.\n"
    f"3. Compare and list missing programming languages that are required in the job description but not in the resume.\n"
    f"4. Compare and list missing soft skills required in the job description but not in the resume.\n"
    f"5. Check overall compatibility between the resume and the job description (high, medium, low).\n"
    f"6. Provide improvement suggestions to increase compatibility with the job role.\n\n"
    f"Provide a structured response in bullet points with short but clear analysis."
)


ollama_payload = {
    "model": "llama2",
    "prompt": ollama_prompt,
    "stream": False
}

# Step 5: Send Request to Ollama
response = requests.post(OLLAMA_API_URL, json=ollama_payload)

if response.status_code == 200:
    result = response.json()
    compatibility_analysis = result.get('response', 'No response received')

    print("\n🔍 **Compatibility Analysis:**")
    print(compatibility_analysis)
else:
    print("\n❌ Error: Failed to process resume for job review.")


📂 Please upload your resume (PDF format)...
